# SHA 1

In [42]:
from functools import reduce

class SHA1:
    def __init__(self):
        self.H = [0x67452301, 0xEFCDAB89, 0x98BADCFE, 0x10325476, 0xC3D2E1F0]
    
    def update(self, stream):
        stream = self._padding(stream)
        stream = self._prepare(stream)

        for block in stream:
            self._process_block(block)
    
    def _padding(self, stream):
        l = len(stream)  # Bytes
        hl = [int((hex(l*8)[2:]).rjust(16, '0')[i:i+2], 16)
              for i in range(0, 16, 2)]

        l0 = (56 - l) % 64
        if not l0:
            l0 = 64

        if isinstance(stream, str):
            stream += chr(0b10000000)
            stream += chr(0)*(l0-1)
            for a in hl:
                stream += chr(a)
        elif isinstance(stream, bytes):
            stream += bytes([0b10000000])
            stream += bytes(l0-1)
            stream += bytes(hl)

        return stream
    
    def _prepare(self, stream):
        M = []
        n_blocks = len(stream) // 64

        stream = bytearray(stream, 'utf8')

        for i in range(n_blocks):  # 64 Bytes per Block
            m = []

            for j in range(16):  # 16 Words per Block
                n = 0
                for k in range(4):  # 4 Bytes per Word
                    n <<= 8
                    n += stream[i*64 + j*4 + k]

                m.append(n)

            M.append(m[:])

        return M
    
    def _process_block(self, block):
        MASK = 2**32-1

        W = block[:]
        for t in range(16, 80):
            W.append(self._ROTL(1, (W[t-3] ^ W[t-8] ^ W[t-14] ^ W[t-16]))
                     & MASK)

        a, b, c, d, e = self.H[:]

        for t in range(80):
            if t <= 19:
                K = 0x5a827999
                f = (b & c) ^ (~b & d)
            elif t <= 39:
                K = 0x6ed9eba1
                f = b ^ c ^ d
            elif t <= 59:
                K = 0x8f1bbcdc
                f = (b & c) ^ (b & d) ^ (c & d)
            else:
                K = 0xca62c1d6
                f = b ^ c ^ d

            T = ((self._ROTL(5, a) + f + e + K + W[t]) & MASK)
            e = d
            d = c
            c = self._ROTL(30, b) & MASK
            b = a
            a = T

            #SHA1.debug_print(t, a,b,c,d,e)

        self.H[0] = (a + self.H[0]) & MASK
        self.H[1] = (b + self.H[1]) & MASK
        self.H[2] = (c + self.H[2]) & MASK
        self.H[3] = (d + self.H[3]) & MASK
        self.H[4] = (e + self.H[4]) & MASK
    
    def _ROTL(self, n, x, w=32):
        return ((x << n) | (x >> w - n))

    def hexdigest(self):
        s = ''
        for h in self.H:
            s += (hex(h)[2:]).rjust(8, '0')
        return s

# Table encoding

In [137]:
def simple_code(message, w):
    new_m = list()
    for i in range(w):
        new_m += message[i::w]
    return ''.join(new_m)

def simple_decode(message, w):
    new_m = list()
    last_l = len(message) % w
    for i in range(last_l):
        new_m.append(message[(len(message) // w+1)*i:(len(message) // w+1)*(i+1)])
    start_id = last_l * (len(message) // w+1)
    for i in range(last_l,w):
        new_m.append(message[start_id+(len(message) // w)*(i-last_l):start_id+(len(message) // w)*(i-last_l+1)])
    out = list()
    for i in range(len(new_m[0])):
        for j in range(len(new_m)):
            out += new_m[j][i:i+1]
    return ''.join(out)

# RSA

In [157]:
import Crypto
from Crypto.PublicKey import RSA
from Crypto import Random
import ast

random_generator = Random.new().read
key = RSA.generate(1024, random_generator)
publickey = key.publickey()

# Sending file hash

In [158]:
with open("capital.txt") as f:
    file = f.read()

SHA = h = SHA1()
for chank in [file[i:i+256] for i in range(0, len(file),256)]:
    h.update(chank)
hex_sha = h.hexdigest()
print("SHA1: ",hex_sha)

hex_enc_1 = simple_code(hex_sha, 13)
print("ENC SYNC: ",hex_enc_1)

hex_enc_2 = publickey.encrypt(hex_enc_1.encode('utf-8'), 32)
print("ENC ASYNC: ",hex_enc_2, "\n\n")

hex_dec_2 = key.decrypt(ast.literal_eval(str(hex_enc_2))).decode('utf-8')
print("DEC ASYNC: ",hex_dec_2)

hex_dec_1 = simple_decode(hex_dec_2, 13)
print("DEC SYNC: ",hex_dec_1)

SHA1:  f800d5f4c9d311574fb6d20930dac66947972e44
ENC SYNC:  f1d485a07c046df65b9f644d7c29907d9233e104
ENC ASYNC:  (b"\x8d\x93K`\xee\xb0\xa18\x91\xb7\xb3\xf9v\xf4\xdd\xce\x81\xc4\xc3.~\xec\xd3\xb7W\xd9\x17\xc4\xbat\xfd\xa1\x00\x07\xa4.S+\xc9SW\xa5\x06\xdd\x1c4?\xd9E\xeba\xb5\xf9\xdd\xcdB\xc2\x94O\xd1q\xb6\x02&\x04\x10\xe7\xaa\x94\x8d\xdd2\xc9+\xd5\xad\xf5$\xba\xb6\x04\xa3\x0b\nC\x81\xeaA'\xbc\xc8r9\x17\xcc\xb4\x12]l\xa6\xb9l\x91\x86|\xf3_\x88lyj\x0cBB\xa7\x14\xe1\xad\xd0\x8d\x04\xfe\x8a\x00\xc4\n\x16\r",) 


DEC ASYNC:  f1d485a07c046df65b9f644d7c29907d9233e104
DEC SYNC:  f800d5f4c9d311574fb6d20930dac66947972e44


# Birthday attack

![title](img/11.PNG)
![title](img/12.PNG)
![title](img/13.PNG)
![title](img/14.PNG)